In [37]:
from pprint import pprint

Many graph problems can be solved using **backtrack** (try one path, if that doesn't work, return from the function call and try the next path).

Tutorial: https://leetcode.com/explore/learn/card/recursion-ii/472/backtracking/2793/
Problems:
- Word Search (in matrix)
- Word Search II （early exit in backtrack ? Store all words in a **trie** to know if our current path is any word's prefix => if not, then early exit.)
- Robot Cleaner
- N-Queens
- Word Break II


### Word Search in Matrix
https://leetcode.com/problems/word-search/

Backtrack 

In [43]:
# Smarter: Backtracking
def exist(board, word: str) -> bool:
    rows, cols = len(board), len(board[0])

    def dfs(row, col, index):
        if index == len(word)-1:
            return True
        if word[index] != board[row][col]:
            return False

        # *Backtrack*: 
        original = board[row][col]
        board[row][col]  = '*'
        found = False
        for nextRow, nextCol in [(row+1,col), (row-1,col), (row,col+1), (row,col-1)]:
            if 0 <= nextRow < rows and 0 <= nextCol < cols and word[index+1] == board[nextRow][nextCol]:
                found = found or dfs(nextRow, nextCol, index+1)
        board[row][col] = original

        return found

    ## Main Code
    for row in range(rows):
        for col in range(cols):
            if board[row][col] == word[0]:
                found = dfs(row, col, 0)
                if found:
                    return True
    return False


In [41]:
# DFS with separated `visited` array ---- one for each stack element. 
def exist(board, word: str) -> bool:
    if not word:
        return 

    """
    DFS starting on a matching entry (==word[0])
    Only go to the next matching entry that's not visited & within the board
    Don't go back. 


    Tricky case:
    [
    ["A","B","C","E"],
    ["S","F","E","S"],
    ["A","D","E","E"]]

    "ABCESEEEFS"
         ^
    At the rightmost "S", going down would work, but going left would not, 
    even though both next moves are "E".


    """
    height, width = len(board), len(board[0])

    def dfs(rowS, colS):
        stack = deque()
        stack.append((rowS, colS, 0, set()))   
        # row, col, index in word, VISITED only for this path
        while stack:
            row, col, index, visited = stack.pop()
            if index == len(word)-1:
                return True
            visited.add((row, col))

            for nextRow, nextCol in [(row,col+1), (row,col-1), (row+1,col), (row-1,col)]:
                # Valid to visit (within the board & not visited)
                if 0 <= nextRow < height and 0 <= nextCol < width and (nextRow,nextCol) not in visited:
                    # Matching next index? 
                    if word[index+1] == board[nextRow][nextCol]:
                        stack.append((nextRow, nextCol, index+1, visited.copy()))





    ## Main code                     
    for row in range(height):
        for col in range(width):
            if board[row][col] == word[0]:
                found = dfs(row, col)
                if found:
                    return True
    return False

# Takes too long
        

### Word Search II
Given a `board` of chars and a list of strings `words`, return all words on board. 

In [9]:

board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]]
pprint(board)

[['o', 'a', 'a', 'n'],
 ['e', 't', 'a', 'e'],
 ['i', 'h', 'k', 'r'],
 ['i', 'f', 'l', 'v']]


In [15]:
len([char for row in board for char in row])


16

### Robot Cleaner
https://leetcode.com/problems/robot-room-cleaner/

Spiral Backtracking !

### N-Queen
Backtrack. 

In [76]:
class NQueen:
    DEBUG = False
    def totalNQueens(self, n: int) -> int:
        '''
        Backtrack: Try place a queen at each row at (row,col) and mark all her attacking zones. 
        Then go try the next row (with backtrack). 
        
        If ^ doesn't work out at (row,col), revert the change and try (row,col+1)
        
        We'll go down each row, and try the columns. When we reach the bottom row and 
        placed a queen there, we found one solution.
        '''
        self.count = 0
        self.board = [[set() for _ in range(n)] for _ in range(n)]
        self.queenBoard = [['.' for _ in range(n)] for _ in range(n)]
#         print(self.queenBoard)
        row = 0
        
        def notUnderAttack(row,col):
            cell = self.board[row][col]  # set of queens (from each row) attacking it 
            return len(cell) == 0
        
        def placeQueen(row,col):
            '''
            Place queen at (row,col). Mark all its attackable zone as `row`. 
            Each queen currently on board can be identified by her row number.
            '''
            self.queenBoard[row][col] = 'Q'
            
            # Add the (row)-th queen as an attacker: 
            # Horizontal
            for attackCol in range(n):
                self.board[row][attackCol].add(row)
            # Vertical
            for attackRow in range(n):
                self.board[attackRow][col].add(row)
            # Up V
            for d in range(1, row+1):
                if col - d >= 0: # stay in the width
                    self.board[row-d][col-d].add(row)
                if col + d < n:
                    self.board[row-d][col+d].add(row)
            # Down V
            for d in range(1, n-row):
                if col - d >= 0:
                    self.board[row+d][col-d].add(row)
                if col + d < n:
                    self.board[row+d][col+d].add(row)
            
            if self.DEBUG:
                print(f"Placed queen at ({row},{col})")
                pprint(self.board)
            
        def removeQueen(row,col):
            '''Remove the queen here at (row,col).
            The queen is identified by her row number.
            '''
            self.queenBoard[row][col] = '.'
            
            # Remove the (row)-th queen as an attacker: 
            # !! Don't remove too many times at (row,col)
            # Horizontal
            for attackCol in range(n):
                self.board[row][attackCol].remove(row)
            # Vertical
            for attackRow in range(n):
                self.board[attackRow][col].discard(row)
            # Up V
            for d in range(1, row+1):
                if col - d >= 0: # stay in the width
                    self.board[row-d][col-d].discard(row)
                if col + d < n:
                    self.board[row-d][col+d].discard(row)
            # Down V
            for d in range(1, n-row):
                if col - d >= 0:
                    self.board[row+d][col-d].discard(row)
                if col + d < n:
                    self.board[row+d][col+d].discard(row)

            if self.DEBUG:
                print(f"Removed queen at ({row},{col})")
                pprint(self.board)
                
        def backtrack(row):
            '''For each row, must place a queen at (row,col) for some col. 
            Backtrack if doesnt work out'''
            for col in range(n):
                if notUnderAttack(row,col):  # can place queen
                    # Base Case: reaches bottom row => Found answer
                    if row == n-1:  
                        self.count += 1
                        self.queenBoard[row][col] = 'Q'
                        self.allValidBoards.append([''.join(boardRow) for boardRow in self.queenBoard])
                        self.queenBoard[row][col] = '.'
                        return
                    
                    # Place the queen here
                    placeQueen(row,col)
                    
                    # Backtrack
                    backtrack(row+1)

                    # Revert the change and try other column
                    removeQueen(row,col)
                
            # If I cannot place a queen at the current row => Dead end. 
            # Return and backtrack to try other plans. 
        
        ## Main Code
        self.allValidBoards = []
        backtrack(0)
        return self.count, self.allValidBoards

s = NQueen()
for n in range(4,5):
    print(f"N={n}")
    count, allValid = s.totalNQueens(n)
    print(f"{count} valid boards: ")
    pprint(allValid)
    print()
    

    

N=4
2 valid boards: 
[['.Q..', '...Q', 'Q...', '..Q.'], ['..Q.', 'Q...', '...Q', '.Q..']]



Time Complexity: $O(N^N) \times 4N$
                
                      ^place and remove queen

Space: $O(N^N)$

Some optimization in setting `placeQueen()`, `removeQueen()`, and `isUnderAttack()`:

(originally they were $O(4N)$, $O(4N)$, and $O(1)$, respectively.

To **Place Queen** at `(row,col)`:
- Row used: After placing the queen, we move down to the next row. Good
- Col used: Record this queen's `col` in a set `cols`. 

>  Cool stuffs for diagonals 😱:
> - Every entry at `(row,col)` in the same main-diagonal has the same `row-col`
> - Every entry at `(row,col)` in the same anti-diagonal has the same `row+col`
- Main diagonal used: Record this queen's `row-col` in a set `mainDiagonals`.
- Anti diagonal used: Record this queen's `row+col` in a set `antiDiagonals`.


To **Remove Queen** at `(row,col)`：
- Row: Nothing to do. Just return from this function and backtrack to previous row.
- Col: remove `col` from `cols`.
- Main diagonal: Remove `row-col` from `mainDiagonals`.
- Anti diagonal: Remove `row+col` from `antiDiagonals`.

Check **isUnderAttack** at `(row,col)`:
- Row: it's a new row so we're good.
- Col: check if `col` is in `cols`
- Main diagonal: check if `row-col` is in `mainDiagonals`.
- Anti diagonal: check if `row+col` is in `mainDiagonals`.

Runtime for all 3 above actions is constant time $O(4)$!



## Combination Sum problems

combination 1,2,3...  Closest combination sum.

In [84]:
list(range(1,10))[-3:]

[7, 8, 9]